In [1]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

Things to think about:
* what if control day not in temperature data set? (solution is to add more temperature data)
* what if control day is actually a heat wave?
* what if control day is close to a heat wave?

In [22]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [23]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/brazil/THI_brazil_90_2022.csv')

In [24]:
heatwaves['month'].unique()

array([ 1,  2, 12, 11,  3])

In [25]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [26]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [27]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [28]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [29]:
heat_days = heat_days.to_dict()['datetime']

In [30]:
heatwaves_active.reset_index(inplace=True)

In [31]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/brazil/daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df.month.unique()
full_df = full_df[~full_df['month'].isin([9, 10, 4])]

full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [32]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [33]:
heatwaves_active['num_iters'] = 'test'

In [34]:

for index, row in heatwaves_active.iterrows():
    subset = full_df[full_df['location'] == row['location']]
    control_day = heatwaves_active.iloc[index, -3]
    i = 0
    try:
        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):


            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        control_day = heatwaves_active.iloc[index, -3]

        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]
            
    print(index / len(heatwaves_active))
            


0.0
2.8718302173975473e-05
5.7436604347950946e-05
8.615490652192643e-05
0.00011487320869590189
0.00014359151086987737
0.00017230981304385286
0.00020102811521782832
0.00022974641739180378
0.0002584647195657793
0.00028718302173975474
0.0003159013239137302
0.0003446196260877057
0.0003733379282616812
0.00040205623043565664
0.0004307745326096321
0.00045949283478360757
0.0004882111369575831
0.0005169294391315585
0.000545647741305534
0.0005743660434795095
0.0006030843456534849
0.0006318026478274604
0.000660520950001436
0.0006892392521754114
0.0007179575543493869
0.0007466758565233624
0.0007753941586973378
0.0008041124608713133
0.0008328307630452887
0.0008615490652192642
0.0008902673673932397
0.0009189856695672151
0.0009477039717411907
0.0009764222739151662
0.0010051405760891415
0.001033858878263117
0.0010625771804370927
0.001091295482611068
0.0011200137847850436
0.001148732086959019
0.0011774503891329945
0.0012061686913069699
0.0012348869934809454
0.0012636052956549208
0.0012923235978288964
0

In [35]:
heatwaves_active['control_day'].min()

Timestamp('2021-11-01 00:00:00')

In [38]:
heatwaves_active[['datetime', 'control_day']]

,datetime,control_day
0,2022-01-12,2022-01-05
1,2022-01-13,2022-01-06
2,2022-02-26,2022-02-12
3,2022-02-27,2022-02-06
4,2021-12-14,2021-12-07
...,...,...
34816,2022-02-26,2022-01-29
34817,2022-02-25,2022-02-18
34818,2022-02-26,2022-02-12
34819,2022-02-25,2022-02-18


In [45]:
valid = heatwaves_active.merge(full_df[['location', 'datetime', 'temp_pct']], left_on=['location', 'control_day'], right_on=['location', 'datetime'])

In [48]:
valid['temp_pct'].describe()

count    34821.000000
mean         0.557720
std          0.169046
min          0.201058
25%          0.420635
50%          0.575397
75%          0.712963
max          0.798942
Name: temp_pct, dtype: float64

In [39]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/brazil/THI_heatwave_control_90.csv')